In [92]:
import keras.metrics
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras import models, layers
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Reshape, BatchNormalization, LeakyReLU
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import keras
import tensorflow as tf
import os
from sklearn.utils import class_weight

In [93]:
def windowing(x, y, window_size, overlap):
    windows = []
    y_labels = []

    step_size = window_size - overlap

    for i in range(0, len(x) - window_size + 1, step_size):
        window = x.iloc[i:(i + window_size), :]
        windows.append(window)

        if np.sum(y[i:i + window_size] == 2) >= 50:
            y_labels.append(1)
        else:
            y_labels.append(0)

    return np.array(windows), np.array(y_labels)

In [94]:
data_path = '../../dataSets/SisFall_labelled'
dfs = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        file_path = os.path.join(root, file)
        dfs.append(pd.read_csv(file_path, index_col=0))

In [95]:
window_size = 200
overlap = 100

In [96]:
windowed_x = []
windowed_y = []
for i, dataset in enumerate(dfs):
    x_w, y_w = windowing(dataset.drop(columns = ['fall']), dataset['fall'], window_size, overlap)
    windowed_x.extend(x_w)
    windowed_y.extend(y_w)

In [97]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(windowed_x)):
    if np.random.rand(1)[0] < 0.2:
        x_test.append(windowed_x[i])
        y_test.append(windowed_y[i])
    else:
        x_train.append(windowed_x[i])

        y_train.append(windowed_y[i])

In [99]:
scaler = MinMaxScaler()

x_train = np.array(x_train)
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)

x_test = np.array(x_test)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [100]:
y_trainCat = to_categorical(y_train, num_classes=2)
input_shape = (200, 3)

tried out class weights but did not improve results

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y_train),y = y_train)
weight = {i : class_weights[i] for i in range(len(class_weights))}

trying out different models, but best one is still the first one

In [101]:
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=4, activation='relu', input_shape=input, padding='same'))

model.add(layers.BatchNormalization())

model.add(layers.MaxPooling1D(pool_size=4, strides=4))

model.add(layers.Conv1D(12, kernel_size=4, activation='relu',padding='same'))

model.add(layers.BatchNormalization())

model.add(layers.MaxPooling1D(pool_size=4, strides=4))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Dropout(0.3))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.AUC()])

In [ ]:
model = models.Sequential()
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(200,3)))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model = models.Sequential()

# Convolutional layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(200, 3)))
model.add(MaxPooling1D(pool_size=2))

# LSTM layer
model.add(LSTM(20, activation='relu'))

# Dense layer for classification
model.add(Dense(2, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#other metric: metrics=[tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives(),'accuracy'] 

In [102]:
batch = 80
epochs = 40
iterationsPerEp = x_train.shape[0] // batch
model.fit(x_train, y_trainCat, batch_size=batch, epochs=epochs, steps_per_epoch=iterationsPerEp)

Epoch 1/40
1530/1530 [==============================] - 8s 5ms/step - loss: 0.0932 - accuracy: 0.9737 - auc_5: 0.9928
Epoch 2/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0397 - accuracy: 0.9870 - auc_5: 0.9983
Epoch 3/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0336 - accuracy: 0.9876 - auc_5: 0.9989
Epoch 4/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0315 - accuracy: 0.9882 - auc_5: 0.9991
Epoch 5/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0296 - accuracy: 0.9888 - auc_5: 0.9991
Epoch 6/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0282 - accuracy: 0.9895 - auc_5: 0.9993
Epoch 7/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0273 - accuracy: 0.9897 - auc_5: 0.9992
Epoch 8/40
1530/1530 [==============================] - 6s 4ms/step - loss: 0.0261 - accuracy: 0.9900 - auc_5: 0.9994
Epoch 9/40
1530/1530 [==============================] - 

In [103]:
predictions = model.predict(x_test)
yPredClasses = np.argmax(predictions, axis=1)
confMat = confusion_matrix(y_test, yPredClasses)
confMat

962/962 [==============================] - 1s 648us/step


array([[30148,    61],
       [  242,   302]])

In [104]:
print('Accuracy', accuracy_score(y_test, yPredClasses))
print('F1-Score', f1_score(y_test, yPredClasses))
print('AUC', roc_auc_score(y_test, yPredClasses))

Accuracy 0.9901473027021754
F1-Score 0.6659316427783903
AUC 0.7765638965209044
